In [1]:
import numpy as np
import torch
from preprocess.dataset import get_MNIST,get_dataset,get_handler
from models.model import Model
# from al_methods.least_confidence import LeastConfidence
# from ssl_methods.semi_fixmatch import fixmatch
import models
from torchvision import transforms
from framework.framework2 import Framework2
import torch.nn as nn
import time
from torch.utils.data import DataLoader
from scipy.spatial.distance import jensenshannon

In [2]:
# in the main 
X_tr, Y_tr, X_te, Y_te = get_dataset("Mnist", "./datasets")

In [3]:
# in the main 
if type(X_tr) is list:
    X_tr = np.array(X_tr)
    Y_tr = torch.tensor(np.array(Y_tr))
    X_te = np.array(X_te)
    Y_te = torch.tensor(np.array(Y_te))

if type(X_tr[0]) is not np.ndarray:
    X_tr = X_tr.numpy()
    X_te = X_te.numpy()
    

In [4]:
# in the main
n_pool = len(Y_tr)
n_test = len(Y_te)
#in the main
handler = get_handler("mnist")
# main or framewrok to see
nEnd=50 # total number to query 
nQuery=1 # nombre of points to query in batch 
nStart=1 # nbre of points to start
NUM_INIT_LB = int(nStart*n_pool/100)
NUM_QUERY = int(nQuery*n_pool/100) if nStart!= 100 else 0
NUM_ROUND = int((int(nEnd*n_pool/100) - NUM_INIT_LB)/ NUM_QUERY) if nStart!= 100 else 0
if NUM_QUERY != 0:
    if (int(nEnd*n_pool/100) - NUM_INIT_LB)% NUM_QUERY != 0:
        NUM_ROUND += 1
print(NUM_INIT_LB,NUM_ROUND,NUM_QUERY)

600 49 600


# Model

In [5]:

model=Model('efficientnet_b7').get_model()
model


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
            (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormAct

In [6]:
model.features = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # Conversion en 3 canaux
model.fc = torch.nn.Linear(2048, 10)  # Modifier la couche de classification pour 10 classes


In [7]:
model

EfficientNet(
  (features): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (avgpool): AdaptiveAvgPool2d(output_size=1)
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=True)
    (1): Linear(in_features=2560, out_features=1000, bias=True)
  )
  (fc): Linear(in_features=2048, out_features=10, bias=True)
)

In [8]:
# in the main file
idxs_lb = np.zeros(n_pool, dtype=bool)
idxs_lb
# in the main file
idxs_tmp = np.arange(n_pool)
idxs_tmp
np.random.shuffle(idxs_tmp)
idxs_tmp
# in the main file
idxs_lb[idxs_tmp[:NUM_INIT_LB]] = True
idxs_lb

array([False, False, False, ..., False, False, False])

In [9]:
len(idxs_lb)

60000

In [10]:
len(idxs_lb[idxs_lb==True])

600

In [11]:
args_pool = {'mnist':
                { 
                 'n_class':10,
                 'channels':1,
                 'size': 28,
                 'transform_tr': transforms.Compose([
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'transform_te': transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'loader_tr_args':{'batch_size': 128, 'num_workers': 8},
                 'loader_te_args':{'batch_size': 1024, 'num_workers': 8},
                 'normalize':{'mean': (0.1307,), 'std': (0.3081,)},
                },
            
        }

In [12]:
class Args:
    def __init__(self,n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize):
        self.n_class=n_class
        self.img_size=img_size
        self.channels=channels
        self.transform_tr=transform_tr
        self.transform_te=transform_te
        self.loader_tr_args=loader_tr_args
        self.loader_te_args=loader_te_args
        self.normalize=normalize
        self.dataset='mnist'
        self.save_path='./save'
        self.model='ResNet50'
        self.lr=0.1
        self.schedule = [20, 40]
        self.momentum=0.9
        self.gammas=[0.1,0.1]
        self.strategy='framwork1'
        self.optimizer='SGD'
        self.save_model=False

In [13]:
dataset_args = args_pool["mnist"]
n_class = dataset_args['n_class']
img_size = dataset_args['size']
channels = dataset_args['channels']
transform_tr = dataset_args['transform_tr']
transform_te = dataset_args['transform_te']
loader_tr_args = dataset_args['loader_tr_args']
loader_te_args = dataset_args['loader_te_args']
normalize = dataset_args['normalize']

In [14]:
args=Args(n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize)

In [15]:
# al_strat=RandomSampling(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args)
# ssl_strat=fixmatch(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args)

In [16]:
framework_2= Framework2(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args,LeastConfidence,fixmatch)

NameError: name 'LeastConfidence' is not defined

In [ ]:
framework_2.train(2e-3,10)

## Data augementation

In [ ]:
# # Create a weakly data augmentation transform
# weak_transform = transforms.Compose([
#     transforms.RandomCrop(14),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor()
# ])

# # Create a strongly data augmentation transform
# strong_transform = transforms.Compose([
#     transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
#     transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 0.1)),
#     transforms.RandomPerspective(),
#     transforms.ToTensor()
# ])


In [ ]:
# # Find indices where the boolean matrix is False
# false_indices = np.argwhere(framework_2.idxs_lb == False).squeeze()

In [ ]:
# U_X_tr=X_tr[false_indices]
# U_Y_tr=Y_tr[false_indices]

In [ ]:
# unlabeled_data_weak=DataLoader(handler(U_X_tr,U_Y_tr,transform=weak_transform))
# unlabeled_data_strong=DataLoader(handler(U_X_tr,U_Y_tr,transform=strong_transform))

### weakly augmented

In [ ]:
# net=framework_2.net.eval()
# predictions_weak = []
# with torch.no_grad():
#     for x, _,_ in unlabeled_data_weak:
#         # Move the batch to the appropriate device (CPU or GPU)
#         x= x.to(framework_2.device)  # device could be 'cuda' or 'cpu'
        
#         # Forward pass to obtain predictions
#         batch_predictions = net(x)
        
#         # Append batch predictions to the list
#         predictions_weak.append(batch_predictions.cpu().numpy())  # Move predictions to CPU and convert to numpy

# # Concatenate predictions from all batches
# predictions_weak = np.concatenate(predictions_weak, axis=0)


In [ ]:
# net=framework_2.net.eval()
# predictions_strong = []
# with torch.no_grad():
#     for x, _,_ in unlabeled_data_strong:
#         # Move the batch to the appropriate device (CPU or GPU)
#         x= x.to(framework_2.device)  # device could be 'cuda' or 'cpu'
        
#         # Forward pass to obtain predictions
#         batch_predictions = net(x)
        
#         # Append batch predictions to the list
#         predictions_strong.append(batch_predictions.cpu().numpy())  # Move predictions to CPU and convert to numpy

# # Concatenate predictions from all batches
# predictions_strong = np.concatenate(predictions_strong, axis=0)


In [ ]:
# jensen=jensenshannon(predictions_weak,predictions_strong,axis=1)

In [ ]:
# np.max(predictions_weak,axis=1)

In [ ]:
# 0.5*(np.max(predictions_weak,axis=1)+np.max(predictions_strong,axis=1))

In [ ]:
# tot=(1-jensen ) *0.5*(np.max(predictions_weak,axis=1)+np.max(predictions_strong,axis=1))

In [ ]:
test_acc=framework_2.predict(X_te,Y_te)

In [ ]:
test_acc

In [ ]:
acc = np.zeros(NUM_ROUND+1)
acc[0] = test_acc

In [ ]:
len(acc)

In [ ]:
for rd in range(1, NUM_ROUND+1):

    if rd%2==0:
        # Al_methods
        print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
        labeled = len(np.arange(n_pool)[idxs_lb])
        if NUM_QUERY > int(nEnd*n_pool/100) - labeled:
            NUM_QUERY = int(nEnd*n_pool/100) - labeled
            
        # query
        ts = time.time()
        output = framework_2.stratAl.query(NUM_QUERY)
        q_idxs = output
        idxs_lb[q_idxs] = True
        te = time.time()
        tp = te - ts
        
        # update
        framework_2.update(idxs_lb)
        best_test_acc = framework_2.train(alpha=2e-3, n_epoch=10)

        t_iter = time.time() - ts
        
        # round accuracy
        # test_acc = strategy.predict(X_te, Y_te)
        acc[rd] = best_test_acc
    else:
        #SSL methods
        
        print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
        labeled = len(np.arange(n_pool)[idxs_lb])
        if NUM_QUERY > int(nEnd*n_pool/100) - labeled:
            NUM_QUERY = int(nEnd*n_pool/100) - labeled
            
        # query
        ts = time.time()

        output = framework_2.stratSSL.query(NUM_QUERY)
        q_idxs = output
        idxs_lb[q_idxs] = True
        te = time.time()
        tp = te - ts
        
        # update
        framework_2.update(idxs_lb)
        best_test_acc = framework_2.stratSSL.train(alpha=2e-3, n_epoch=10)

        t_iter = time.time() - ts
        
        # round accuracy
        # test_acc = strategy.predict(X_te, Y_te)
        acc[rd] = best_test_acc


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')
import seaborn as sns
sns.set_theme()

plt.figure(figsize=(10,5))
sns.lineplot(data=acc,label='Accuracy Line')
plt.xlabel('Rounds')
plt.ylabel('Accuracy')

plt.yticks(np.arange(0.1,1,0.05))

# Add a title

plt.title('Accuracy Over Rounds for the framwork 1 (least+semifixMatch)')
# Add a grid
plt.grid(True)
plt.show()

In [ ]:
len(acc)

In [ ]:
np.save("least_fix_acc.npy",acc)

In [ ]:
import gc

In [ ]:
gc.collect()